# FINAL PROJECT 2024
The goal of this project is to implement trending topic detection algorithms in X messages, as proposed in the article "Sensing Trending Topics in Twitter" by L. M. Aiello et al., IEEE Transactions on Multimedia, vol. 15, no. 6, pp. 1268-1282, Oct. 2013, doi: 10.1109/TMM.2013.2265080.

•	The suggested Databricks runtime version is the 12.2 LTS ML (Scala 2.12, Spark 3.3.2) instead the Standard version.

•	You have to follow the instructions provided in this webpage to install the SparkNLP library in Databricks, in order to be able to use it in that environment:

https://sparknlp.org/docs/en/install#databricks-support


In [0]:
from pyspark.sql.functions import col, udf, explode, from_json
from pyspark.sql.types import *
import sparknlp
import json
import urllib.request, json 
import nltk

from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import (
    LanguageDetectorDL
)
import pyspark.sql.functions as F

### Function to retrieve data
In the following function we are retrieving the data (tweets) based on time parameters.

You must specify the start date (year, month, day, hour. minute) and the time lenght of the data to be retrieved

Here we create the url for the given parameters and then proceed to make a request




In [0]:
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import urllib.request
import os

def get_tweets(start_year=2019, start_month=8, start_day=1, start_hour=2, start_minute=0, minutes_length=5):
    """
    Fetch tweets from the given URL based on the specified parameters and return a filtered DataFrame.

    Parameters:
        start_year (int): The starting year for fetching tweets.
        start_month (int): The starting month for fetching tweets.
        start_day (int): The starting day for fetching tweets.
        start_hour (int): The starting hour for fetching tweets.
        start_minute (int): The starting minute for fetching tweets.
        minutes_length (int): The total duration in minutes to fetch tweets.

    Returns:
        pyspark.sql.DataFrame: Filtered DataFrame with selected tweet fields.
    """
    # Initialize SparkSession
    spark = SparkSession.builder.appName("GetTweets").getOrCreate()
    
    # Define the maximum duration for a single request
    MAX_MINUTES = 30

    # Start time
    start_time = datetime(start_year, start_month, start_day, start_hour, start_minute)
    end_time = start_time + timedelta(minutes=minutes_length)

    # Initialize a list to collect data from all chunks
    all_data = []

    # Loop through the time intervals in chunks of MAX_MINUTES
    current_time = start_time
    while current_time < end_time:
        # Calculate the duration for this request
        request_minutes = min(MAX_MINUTES, (end_time - current_time).seconds // 60)
        
        # Construct the URL for the current chunk
        datafile = (
            f"http://mcomputing.tsc.uc3m.es/get_tweets.php?start_year={current_time.year}"
            f"&start_month={current_time.month:02d}&start_day={current_time.day:02d}"
            f"&start_hour={current_time.hour:02d}&start_minute={current_time.minute:02d}"
            f"&minutes_length={request_minutes}"
        )
        
        try:
            # Download the JSON data for the current chunk
            with urllib.request.urlopen(datafile) as url:
                data = url.read().decode()
                all_data.append(data)

        except Exception as e:
            print(f"Failed to fetch data for interval starting at {current_time}: {e}")

        # Increment the current time
        current_time += timedelta(minutes=request_minutes)

    # If no data was fetched, display a message and return
    if not all_data:
        print("No data was fetched for the given time range.")
        return None

    # Save all data into a single temporary file
    temp_file_path = "/tmp/temporal.json"
    with open(temp_file_path, "w") as f:
        for chunk in all_data:
            f.write(chunk + "\n")  # Write each chunk on a new line

    # Read the combined JSON file into a Spark DataFrame
    tweet_df = spark.read.json(f"file://{temp_file_path}")
    
    # Filter and select the desired fields
    filtered_df = tweet_df.select("created_at", "user.id", "user.name", "user.screen_name", "user.lang", "text")

    return filtered_df



In [0]:
tweets = get_tweets()

In [0]:
tweets.count()

Out[10]: 11206

In [0]:
tweets.columns

Out[11]: ['created_at', 'id', 'name', 'screen_name', 'lang', 'text']

In [0]:
display(tweets.select("created_at","id","text").head(10))

created_at,id,text
Thu Aug 01 08:00:00 +0000 2019,2506848924,Чому не варто терти очі кулаками: думка експертів
Thu Aug 01 08:00:00 +0000 2019,1110566818576388096,kamala harris fucked up bgt parah
Thu Aug 01 08:00:00 +0000 2019,781056205179326464,RT @mgmgnet: 朝早くドトールに行ったら白髪のおじさまが「このタピ…ってやつはあるかい？」「俺みたいなんが飲んだら喉につまらせるかね？」「楽しみだなぁ、テレビで見て飲んでみたかったんだ」とニコニコしていて大変よかった ドトールでタピオカする意味ある
Thu Aug 01 08:00:00 +0000 2019,1132529419099095040,RT @niwaniwa_28ko: ポコに「ちょうだい」を教えてみたよ♪(๑’ᵕ’๑)♪ #ニワトリは飼うと可愛い https://t.co/Dbh5X9Ixd7
Thu Aug 01 08:00:00 +0000 2019,1106855239578087424,@elementslover 乗ります！
Thu Aug 01 08:00:00 +0000 2019,613459404,2019-08-01T07:59:15.000Z1156836779833483264
Thu Aug 01 08:00:00 +0000 2019,1388797110,RT @loopsjongin: https://t.co/uzS31kV1mS
null,null,null
Thu Aug 01 08:00:00 +0000 2019,524872493,RT @lament1ess: 手取り23万、食費4万。僕はただ働き続け、気付けば日々弾力を失っていく心がひたすらつらかった。 そしてある朝、かつてあれほどまでに真剣で切実だった思いがきれいに失われていることに僕は気づき、もう限界だと知った時、会社を辞めた。 https://…
Thu Aug 01 08:00:00 +0000 2019,1867085616,@pcyegan wkwkwk sedi bre kita kepisah


### Function performing initial data cleaning
Here, we are cleaning the tweets by converting timestamps, revoming nulls and unwanted characters.

First we will deal with the created_at column to properly convert it into a timestamp format

Then we will remove rows with any null values in the columns of relevance which are created_at and text mainly

For better refinement, we get rid of any characters that arent alphnumeric or _ (or @ # if special_cahracters is set to True), so this way we get rid of ! , etc, and any symbols belonging to a different alphabet. And we convert to lower case. Using it does not change teh rseults much since if we have #love or @mike32 they will simply become love and mike32, in which case if they are repeated often, it will still show up as relevant.

After applying this filter, any rows that have now been emptied, will be removed.

In [0]:
from pyspark.sql import functions as F

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY") # important for the time column conversion


def inital_cleaning_prep(tweets, special_characters=False, column_name='text'):
    """
    Performs initial cleaning on a DataFrame of tweets:
    - Converts `created_at` to a timestamp.
    - Removes rows with null values in `created_at`, `id`, or `text`.
    - Removes symbols and special characters from the `text` column.
    - Converts the `text` column to lowercase.
    - Removes rows where the `text` column is empty after cleaning.

    Parameters:
        tweets (DataFrame): The input PySpark DataFrame.
        column_name (str): The name of the column to clean (default is 'text').

    Returns:
        DataFrame: The cleaned PySpark DataFrame.
    """
    # Step 1: Convert `created_at` to a timestamp
    tweets = tweets.withColumn(
        "created_at",
        F.to_timestamp("created_at", "EEE MMM dd HH:mm:ss Z yyyy")  # Legacy version
    )
    
    # Step 2: Remove rows with null values in `created_at`, `id`, or `text`
    cleaned_tweets = tweets.dropna(subset=["created_at", "id", column_name])

    # Step 3.1: Remove symbols and special characters from the `text` column
    if special_characters:
        cleaned_tweets = cleaned_tweets.withColumn(
            column_name,
            F.regexp_replace(column_name, r"[^\w\s@#]", "")  # Remove all non-word or number characters except whitespace, `@` and `#` 
        )
    else:
        cleaned_tweets = cleaned_tweets.withColumn(
            column_name,
            F.regexp_replace(column_name, r"[^\w\s]", "")  # Remove all non-word or number characters except whitespace
        )

    # Step 3.2: Convert the `text` column to lowercase
    cleaned_tweets = cleaned_tweets.withColumn(
        column_name,
        F.lower(F.col(column_name))  # Convert to lowercase
    )

    # Step 4: Remove rows where the `text` column is empty after cleaning
    cleaned_tweets = cleaned_tweets.filter(F.col(column_name).rlike(r"\S"))  # Keep rows where `text` has non-whitespace characters

    return cleaned_tweets



In [0]:
tweets = inital_cleaning_prep(tweets)

In [0]:
display(tweets.select("created_at","id","text").head(10))

created_at,id,text
2019-08-01T08:00:00.000+0000,1110566818576388096,kamala harris fucked up bgt parah
2019-08-01T08:00:00.000+0000,781056205179326464,rt mgmgnet
2019-08-01T08:00:00.000+0000,1132529419099095040,rt niwaniwa_28ko httpstcodbh5x9ixd7
2019-08-01T08:00:00.000+0000,1106855239578087424,elementslover
2019-08-01T08:00:00.000+0000,613459404,20190801t075915000z1156836779833483264
2019-08-01T08:00:00.000+0000,1388797110,rt loopsjongin httpstcouzs31kv1ms
2019-08-01T08:00:00.000+0000,524872493,rt lament1ess 234 https
2019-08-01T08:00:00.000+0000,1867085616,pcyegan wkwkwk sedi bre kita kepisah
2019-08-01T08:00:00.000+0000,1964776824,rt onthecccccc httpstcosylbr1tm7d
2019-08-01T08:00:00.000+0000,1121798073808613376,rt jnkchanels this is so sad httpstcooqr6apodz1


In [0]:
tweets.count()

Out[19]: 9311

### Function filters English tweets
With this function we filter tweets to retain only those in English, like in the original example provided to us.

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import LanguageDetectorDL
from pyspark.ml import Pipeline

def filter_english_tweets(filteredDF):
    """
    Filters English tweets using Spark NLP's LanguageDetectorDL.

    Args:
    - filteredDF: Spark DataFrame with a `text` column.

    Returns:
    - DataFrame: Filtered DataFrame containing only English tweets with required columns.
    """
    # Step 1: Language Detection Pipeline
    document_assembler = DocumentAssembler() \
        .setInputCol("text") \
        .setOutputCol("document")

    language_detector = LanguageDetectorDL.pretrained() \
        .setInputCols("document") \
        .setOutputCol("language")

    nlp_pipeline = Pipeline(stages=[document_assembler, language_detector])
    detected_languages = nlp_pipeline.fit(filteredDF).transform(filteredDF)

    # Step 2: Filter for English Language
    def is_english(language_data):
        return 'en' in language_data

    udf_is_english = udf(is_english, BooleanType())

    english_tweets = detected_languages.filter(
        udf_is_english("language.result")
    ).select("created_at", "text")  # Select only relevant columns

    return english_tweets



In [0]:
tweets = filter_english_tweets(tweets)

ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
[OK!]


In [0]:
display(tweets.select("created_at","text").head(10))

created_at,text
2019-08-01T08:00:00.000+0000,kamala harris fucked up bgt parah
2019-08-01T08:00:00.000+0000,rt jnkchanels this is so sad httpstcooqr6apodz1
2019-08-01T08:00:00.000+0000,auceiram ako sana kaya lang walang pera wait hanap buhay muna ako
2019-08-01T08:00:00.000+0000,rt sam56786260 zitudiary amen
2019-08-01T08:00:00.000+0000,nerf everywhere
2019-08-01T08:00:00.000+0000,01082019 085801 httpstcombaecjem9y blastfmcountry buck johnson country rockin amp reelin
2019-08-01T08:00:00.000+0000,mayazzam_ almasryalyoum
2019-08-01T08:00:00.000+0000,rt hotladysite you will simply choose a woman and ask to fuck httpstco0g4uladojt httpstcozw8ktl8jkf
2019-08-01T08:00:00.000+0000,thank goodness for waterproof mattress covers
2019-08-01T08:00:00.000+0000,predicar la unitat s incompatible amb dedicar retrets constants als socis que noms faran descarrilar la respos httpstcook3gh2rwff


In [0]:
tweets.count()

Out[23]: 3587

## Data preparation - BNGRAM

### Data preprocessing for BNgram

Now that we have cleared the first steps, getting the data and the initial language filter (plus the extra cleaning of null or empty rows) we will perform further data preparation before applying the topic detection algorithm.

Given that we have chosen to apply BNgram, these steps haing refined for this specific case.

In these stpes we include:

- **Tokenization** - splits text into tokens and removes punctuation, hyphens, and abbreviations.

- **Temporal aggregation** - gropu the documents with the clean text (treated tokens) into superdocuments based on a time window

After this rows which are empty are removed.

In this process there is something specific to **BNGram** worth mentioning. To enhance the performance of BNgram, **NER** (Named Entity Recognition) can be applied. What this will do is extract the relevant words which represent known entities, like names or places, so that they can later on be used in the scoring of the ngrams (groups of words) giving more importance to the ones that are recogniezd as entities, since this will help identify topics. It is worth noting, that we allow the user to apply NER or not, given that adding it increments the running time a lot.

I will also mention here that we had implemented stemming, but it was later removed since, like it is said in the paper, in the case of bngram it worsens the performance.



### Step by step

We will go through the steps, seeing the intermediate results of each part of the preprocessing and algorithm


#### NER

We extract named entities from tweets using a pretrained (NER) pipeline, if requested by the user.

Like we said, if it is implemented it will take some time to run.

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, window, collect_list, lower, size, expr
from sparknlp.annotator import Tokenizer, StopWordsCleaner
from sparknlp.pretrained import PretrainedPipeline

# NER part separate - given that it is a heavy process

def extract_entities(input_df):
    """
    Apply a pretrained NER pipeline to extract entities from text data.

    Parameters:
    - input_df: Spark DataFrame with columns ["created_at", "text"].

    Returns:
    - enriched_df: Spark DataFrame with additional "entities" column.
    """
    # Initialize the pretrained NER pipeline
    ner_pipeline = PretrainedPipeline("onto_recognize_entities_sm", lang="en")

    # Extract entities row by row locally
    enriched_rows = []
    for row in input_df.toLocalIterator():  # Process locally to avoid serialization issues
        ner_result = ner_pipeline.annotate(row["text"])
        enriched_rows.append(Row(created_at=row["created_at"], text=row["text"], entities=ner_result.get("entities", [])))

    # Convert enriched rows back to a DataFrame
    enriched_df = spark.createDataFrame(enriched_rows)

    return enriched_df



We prepare text data for n-gram generation and temporal aggregation, folowing the stpes described above.

In [0]:
def preprocess_for_bngram(input_df, time_window="1 minutes", apply_ner=True):
    """
    Preprocess data for the BNgram algorithm.

    Parameters:
    - input_df: Spark DataFrame containing the raw text data.
    - time_window: Time window for temporal aggregation (default: '5 minutes').
    - apply_ner: Whether to apply the NER step (default: True).

    Returns:
    - aggregated_data: Spark DataFrame with temporally aggregated text data.
    """
    # Apply NER if requested
    if apply_ner:
        input_df = extract_entities(input_df)

    # Step 1: Tokenization
    document_assembler = DocumentAssembler() \
        .setInputCol("text") \
        .setOutputCol("document")

    tokenizer = Tokenizer() \
        .setInputCols(["document"]) \
        .setOutputCol("token")

    stopwords_cleaner = StopWordsCleaner() \
        .setInputCols(["token"]) \
        .setOutputCol("clean_tokens") \
        .setCaseSensitive(False)

    # Create text cleaning pipeline
    text_pipeline = Pipeline(stages=[document_assembler, tokenizer, stopwords_cleaner])
    processed_data = text_pipeline.fit(input_df).transform(input_df)

    # Filter out rows with empty clean_text
    non_empty_text = processed_data.filter(size(col("clean_tokens.result")) > 0)

    # Step 2: Temporal Aggregation
    if apply_ner:
        aggregated_data = non_empty_text.groupBy(window("created_at", time_window)) \
            .agg(collect_list("clean_tokens.result").alias("super_documents"),
                 collect_list("entities").alias("entities_list"))
    else:
        aggregated_data = non_empty_text.groupBy(window("created_at", time_window)) \
            .agg(collect_list("clean_tokens.result").alias("super_documents"))
    # Filter out empty super-documents
    aggregated_data = aggregated_data.filter(size(col("super_documents")) > 0)

    return aggregated_data




In [0]:
aggregated_tweets = preprocess_for_bngram(tweets, time_window='1 minutes', apply_ner=True)

onto_recognize_entities_sm download started this may take some time.
Approx size to download 159 MB
[OK!]


In [0]:
# display(aggregated_tweets.head(10)) # due to the super docs in time aggreagation it takes too long to run otherwse

display(aggregated_tweets.head(1))

window super_documents entities_list List(2019-08-01T08:00:00.000+0000, 2019-08-01T08:01:00.000+0000) List(List(kamala, harris, fucked, bgt, parah), List(rt, jnkchanels, sad, httpstcooqr6apodz1), List(auceiram, ako, sana, kaya, lang, walang, pera, wait, hanap, buhay, muna, ako), List(rt, sam56786260, zitudiary, amen), List(nerf, everywhere), List(01082019, 085801, httpstcombaecjem9y, blastfmcountry, buck, johnson, country, rockin, amp, reelin), List(mayazzam_, almasryalyoum), List(rt, hotladysite, simply, choose, woman, ask, fuck, httpstco0g4uladojt, httpstcozw8ktl8jkf), List(thank, goodness, waterproof, mattress, covers), List(predicar, la, unitat, incompatible, amb, dedicar, retrets, constants, als, socis, que, noms, faran, descarrilar, la, respos, httpstcook3gh2rwff), List(kane, wootten, said, personal, issues, time, incident, httpstco9qmjn7yrig), List(backlash, views, transgender, children, lopez, backtracks, httpstcox8x598ystw), List(httpstcoumtknu4afr, panther, dan, httpstcorftcbca6bl), List(trump, administration, says, set, system, allowing, americans, access, lowercost, prescription, drugs, c, httpstcob4pevdsnhd), List(2016, case, la, manada, spain, set, massive, protests, call, reform, legislation, two, recent, cases, httpstcowbkmyx1ikt), List(consumers, rank, cybersecurity, ahead, price, attractiveness, choosing, primary, retailer, httpstco0ooc2z8fus), List(looking, today, todays, worldwidewebday, httpstcol4n7qsxkv3), List(_, _, _, httpstcomugqt6wpps), List(futureofldns, making, londons, waterways, conference, took, place, 20th, june, 2019, crps, director, susann, httpstcowlvwmojgvb), List(httpstcoaukvtnxkbc, big, sis, takes, squirt, thecadencelux, lillillylitxxx, httpstco3vcgukjzeb), List(road, work, sr, 86ajo, way, reduced, one, lane, way, overnight, week, 9, pm, 5, i1, httpstcogrg7wfxgi0), List(r, xoxox1004, sbryytsyo, httpstcosfcmvctpgi), List(321eshop, 3d, floor, sticker, wall, painting, stone, steps, small, road, bathroom, httpstcooqq7i1xcea), List(use, computer, home, without, help, clicksilver, participant, find, httpstcoummlcxkxin, httpstco3bdnr7cyxv), List(countdown, begun, 3500, attendees, counting, book, seat, today, httpstcofwitohrt7x, httpstco5lozt6cue1), List(new, review, biblio_kitten, enjoyed, fascinating, characters, clever, suspenseful, plot, good, girl, bad, girl, httpstcool4foppary), List(loreal, committed, making, profound, transformation, towards, lowcarbon, business, model, ambitious, httpstcockruj1lvys), List(httpstcorelcl9wkfe), List(cs01081912214, scored, 80, dont, click), List(rt, cheminahsayang, king, httpstcomjmawhsnnb), List(rt, halalcoholism, orange, man, actually, bad, tho), List(rt, cheesewhis, httpstcoxiiy5cn8fv), List(rt, hmpbarlvisitors, beautifully, written, piece, family, experiences, someone, prison, httpstcoha0xbomvas, hear), List(rt, trumpwarroom, kamala, harris, says, know, predators, fact, check, true, top, staffer, 14, years, accused, sexual, harassment), List(subwayyellow), List(rt, asg1956, growth, 8, core, sectors, dips, 02, june, near, 4year, low, times, india, httpstcov7fnelkbco, mr, narendramodi, ke), List(hashtag4, women, vintage, wear, work, elegant, business, party, bodycon, sheath, office, ruffle, female, dress, httpstcoy0utdq0qp9), List(iiateef, httpstcoe3fl5ayymb), List(1st, aug, 2019, holybible, httpstcoarognyesya), List(rayk_ea11r), List(rt, kangminehee, early, po, x1, debut, album, estimasi, harga, rp, 225000, sd, rp, 325000, harga, fix, bisa, diketahui, ketika, barang, sudah, rilis, b), List(rt, gameofryuk007, happybirthday, 23, 2019, 2019, 81, rt), List(rt, tradielover1, wow, nice, piece, meat, httpstcodfnzeprjjm), List(rt, meyou57449329, proudresister, racisttrump), List(rt, iluvmayward1, minsan, still, pinch, na, eto, na, ang, babies, natin, rubbing, elbows, stars, na, idols, lang, din, nila, da), List(rt, meyou57449329, proudresister, racisttrump), List(rt, weverseforbts, weversetrans, v, bts_twt, op, taehyung, still, exercise, days, v, dont, became, sm

### Algorithm implementation - BNGRAM

Now that we have our superdocuments, with the clean tokens per time window (and entities if NER was applied). The next steps for the implementation of the BNgram algorithm are the following:

- **Generate n-grams:** Create n-grams (sequences of `n` words) from `super_documents` in the aggregated data.
- **Compute temporal relevance:** Calculate relevance scores for n-grams over time to identify trends, based on the frequency of occurence in each time window, taking into account the curent and past frequency in each window.
- **Boost named entities:** Boost n-grams containing named entities if `apply_ner=True`, otheriwse the boosted score will be the same as the relevance calculated in the previous step.
- **Cluster n-grams:** Group similar n-grams using KMeans clustering for a certain number of clusters (`num_clusters`), first converting the words into vector embeddings to create numerical features which can be clustered.
- **Rank topics:** In the final stpes we rank n-grams within clusters based on boosted scores and relevance. 

These ranked n-grams per cluster will be the final result. Each cluster is supposed to represent a different group of topics, which are semantically more similar, and in each of these group of topics, the speciic n-grams are ordered in terms of relevance to show the more popular topics in each cluster.

Let's go through each of the functions that compose the overall algorihtm.


We generate n-grams (sequences of n words) from text data.

In [0]:
from pyspark.sql.functions import col, explode, flatten, sequence, concat_ws, expr

def generate_ngrams(aggregated_data, n=5):
    """
    Generate n-grams from aggregated super_documents.

    Parameters:
    - aggregated_data: Spark DataFrame with `super_documents`.
    - n: The size of n-grams to generate.

    Returns:
    - DataFrame with `window` and generated `ngrams`.
    """

    # Flatten super_documents to a single array
    processed_data = aggregated_data.withColumn("flat_documents", flatten(col("super_documents")))

    # Generate n-grams for each document
    ngrams_df = processed_data.withColumn(
        "ngrams",
        explode(
            expr(f"""
                transform(
                    sequence(1, size(flat_documents) - {n} + 1),
                    pos -> concat_ws(" ", slice(flat_documents, pos, {n}))
                )
            """)
        )
    ).select("window", "ngrams")

    return ngrams_df



In [0]:
ngrams_df = generate_ngrams(aggregated_tweets)

In [0]:
ngrams_df.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- ngrams: string (nullable = false)



We calculate a temporal relevance score for each n-gram.

In [0]:
from pyspark.sql.functions import col, avg, count, unix_timestamp, struct

def compute_temporal_relevance(ngrams_df):
    """
    Compute temporal relevance scores for n-grams.

    Parameters:
    - ngrams_df: DataFrame with `ngrams` and `window`.

    Returns:
    - DataFrame with `ngrams`, `window`, `current_count`, `average_past_count`, and `relevance_score`.
    """

    # Convert window.start to a numeric timestamp
    ngrams_df = ngrams_df.withColumn("window_start", unix_timestamp(col("window.start")))

    # Count occurrences of n-grams in each window
    current_counts = ngrams_df.groupBy("window_start", "ngrams").agg(
        count("*").alias("current_count")
    )

    # Calculate average past counts (excluding the current window)
    past_counts = current_counts.groupBy("ngrams").agg(
        avg("current_count").alias("average_past_count")
    )

    # Join current and past counts
    relevance_df = current_counts.join(
        past_counts, on="ngrams", how="left"
    ).withColumn(
        "relevance_score",
        col("current_count") / (col("average_past_count") + 1)  # Avoid division by zero
    )

    return relevance_df



In [0]:
relevance_df = compute_temporal_relevance(ngrams_df)

In [0]:
relevance_df.printSchema()

root
 |-- ngrams: string (nullable = false)
 |-- window_start: long (nullable = true)
 |-- current_count: long (nullable = false)
 |-- average_past_count: double (nullable = true)
 |-- relevance_score: double (nullable = true)



Here we aim to enhance the relevance scores of n-grams containing named entities, when NER is applied

In [0]:
from pyspark.sql.functions import col, struct, from_unixtime, expr, explode, concat

def boost_named_entities(relevance_df, aggregated_data, apply_ner=True, time_window_seconds=300):
    """
    Boost relevance scores for n-grams containing named entities.

    Parameters:
    - relevance_df: DataFrame with computed relevance scores.
    - aggregated_data: Original aggregated data with `entities_list`.
    - apply_ner: Boolean indicating whether to apply NER boosting.
    - time_window_seconds: Window duration in seconds (default: 300 seconds).

    Returns:
    - DataFrame with boosted scores.
    """
    if not apply_ner:
        # If NER is not applied, return relevance_df with a boosted_score column equal to relevance_score
        boosted_df = relevance_df.withColumn("boosted_score", col("relevance_score"))
        return boosted_df

    # Step 1: Explode entities_list into individual entities and ensure the entity is a string
    entities_df = aggregated_data.select(
        "window",
        explode(col("entities_list")).alias("entity")
    ).withColumn(
        "entity", col("entity").cast("string")  # Ensure entity is a string
    )

    # Step 2: Reconstruct `window` column in relevance_df
    relevance_df = relevance_df.withColumn(
        "window",
        struct(
            from_unixtime(col("window_start")).cast("timestamp").alias("start"),
            (from_unixtime(col("window_start")).cast("timestamp") + expr(f"INTERVAL {time_window_seconds} SECONDS")).alias("end")
        )
    )

    # Step 3: Perform the join and calculate boosted scores
    boosted_df = relevance_df.join(
        entities_df, on="window", how="left"
    ).withColumn(
        "boosted_score",
        expr("relevance_score * (case when ngrams like concat('%', entity, '%') then 1.5 else 1 end)")
    )

    return boosted_df



In [0]:
boosted_df = boost_named_entities(relevance_df, aggregated_data= aggregated_tweets, apply_ner = True)

We cluster n-grams into topics using Word2Vec embeddings and KMeans clustering.

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Word2Vec

def cluster_ngrams(boosted_df, num_clusters=5):
    """
    Cluster related n-grams into topics using Word2Vec and KMeans.

    Parameters:
    - boosted_df: DataFrame with boosted relevance scores.
    - num_clusters: Number of clusters for KMeans.

    Returns:
    - DataFrame with clusters assigned to n-grams.
    """
    # Aggregate n-grams into lists for each window
    ngram_docs = boosted_df.groupBy("window_start").agg(
        collect_list("ngrams").alias("ngram_list")
    )

    # Fit Word2Vec model to get embeddings
    word2vec = Word2Vec(vectorSize=50, inputCol="ngram_list", outputCol="features")
    w2v_model = word2vec.fit(ngram_docs)

    # Transform n-grams into embeddings
    ngram_embeddings = w2v_model.transform(ngram_docs)

    # Apply KMeans clustering
    kmeans = KMeans(k=num_clusters, featuresCol="features", predictionCol="cluster")
    kmeans_model = kmeans.fit(ngram_embeddings)

    # Assign clusters
    clustered_df = kmeans_model.transform(ngram_embeddings)

    # Explode ngrams and add cluster labels
    final_df = clustered_df.select(
        "window_start", explode("ngram_list").alias("ngram"), "cluster"
    )

    return final_df


In [0]:
clustered_df = cluster_ngrams(boosted_df)

We rank clusters (topics) based on the average boosted scores of their n-grams.

In [0]:
from pyspark.sql.functions import col, avg, collect_list, desc

def rank_topics(clustered_df, boosted_df):
    """
    Rank topics based on the average boosted scores of their n-grams.

    Parameters:
    - clustered_df: DataFrame with clusters assigned to n-grams.
    - boosted_df: DataFrame with boosted scores for n-grams.

    Returns:
    - DataFrame with ranked topics, their associated n-grams, and average scores.
    """
    # Join clustered_df with boosted_df to include boosted_score
    ranked_df = clustered_df.join(
        boosted_df.select(col("ngrams").alias("ngram"), "boosted_score"),
        on="ngram",
        how="inner"
    )

    # Group by cluster and calculate aggregated information
    ranked_topics = ranked_df.groupBy("cluster").agg(
        collect_list("ngram").alias("ngrams"),
        avg("boosted_score").alias("average_score")
    ).orderBy(desc("average_score"))

    return ranked_topics



In [0]:
ranked_topics = rank_topics(clustered_df, boosted_df)

lets see what the top 3 n-grams per cluster are

In [0]:
from pyspark.sql.functions import col, slice

def get_top_ngrams_per_cluster(ranked_topics, top_n=3):
    """
    Extract the top n-grams for each cluster.
    
    Parameters:
    - ranked_topics: Spark DataFrame containing ranked topics with "ngrams" and "cluster".
    - top_n: Number of top n-grams to extract per cluster.
    
    Returns:
    - Spark DataFrame with top n-grams for each cluster.
    """
    top_n_topics_with_ngrams = ranked_topics.withColumn(
        "top_ngrams", slice(col("ngrams"), 1, top_n)
    ).select("cluster", "top_ngrams", "average_score")
    return top_n_topics_with_ngrams


In [0]:
top_n_topics_with_ngrams = get_top_ngrams_per_cluster(ranked_topics, top_n=3)

display(top_n_topics_with_ngrams)

cluster,top_ngrams,average_score
2,"List(chanyeol sehun_chanyeol exo_sc, think babies mature, opinion httpstcosicxdyj4rc rt)",0.5077807896778362
4,"List(opened gate httpstcoyzlabbgvvb, shouldnt tell someo, 20s baby life)",0.507086928801416
1,"List(way overnight week, face dig face, britains industry gre)",0.5063540922014329
3,"List(chanyeol sehun_chanyeol exo_sc, karlousm damn good, rt peryiat go)",0.5059210868267039
0,"List(hmmmmm food religion, hassystants 12 816, fave look unbuttoned)",0.5046124569067216


if we ignore the cluster they belong to, overall the top n-grmas woudl be the following

In [0]:
from pyspark.sql.functions import explode

def get_top_overall_ngrams(ranked_topics, top_n=5):
    """
    Extract the top n-grams across all clusters.
    
    Parameters:
    - ranked_topics: Spark DataFrame containing ranked topics with "ngrams".
    - top_n: Number of overall top n-grams to extract.
    
    Returns:
    - Spark DataFrame with top n-grams.
    """
    top_ngrams = ranked_topics.select(explode(col("ngrams")).alias("ngram")).limit(top_n)
    return top_ngrams


In [0]:
top_ngrams = get_top_overall_ngrams(ranked_topics, top_n=5)

display(top_ngrams)

ngram
chanyeol sehun_chanyeol exo_sc
think babies mature
opinion httpstcosicxdyj4rc rt
top bgt masih
pickup ssr3 rt


### Final BNgram function

In the next function we integrated all the steps to extract and rank trending topics.

In [0]:
def bngram_algorithm(aggregated_data, n=5, apply_ner=False, num_clusters=5):
    """
    Apply the BNgram algorithm to aggregated data.

    Parameters:
    - aggregated_data: Aggregated Spark DataFrame with `super_documents` and optionally `entities_list`.
    - n: Number of words in the n-grams.
    - apply_ner: Boolean flag indicating whether NER was applied.
    - num_clusters: Number of clusters for KMeans clustering.

    Returns:
    - DataFrame with ranked topics and their n-grams.
    """
    # Step 1: Generate n-grams
    ngrams_df = generate_ngrams(aggregated_data, n)

    # Step 2: Compute temporal relevance scores
    relevance_df = compute_temporal_relevance(ngrams_df)

    # Step 3: Boost named entities (conditionally)
    boosted_df = boost_named_entities(relevance_df, aggregated_data, apply_ner)

    # Step 4: Cluster n-grams
    clustered_df = cluster_ngrams(boosted_df, num_clusters)

    # Step 5: Rank topics
    ranked_topics = rank_topics(clustered_df, boosted_df)

    return ranked_topics



In [0]:
topics = bngram_algorithm(aggregated_tweets, n=5, apply_ner=True, num_clusters=5)

In [0]:
top_topics_per_cluster = get_top_ngrams_per_cluster(topics, top_n=3)

display(top_topics_per_cluster)

cluster,top_ngrams,average_score
2,"List(chanyeol sehun_chanyeol exo_sc, think babies mature, opinion httpstcosicxdyj4rc rt)",0.5077807896778362
4,"List(opened gate httpstcoyzlabbgvvb, shouldnt tell someo, 20s baby life)",0.507086928801416
1,"List(way overnight week, face dig face, britains industry gre)",0.5063540922014329
3,"List(chanyeol sehun_chanyeol exo_sc, karlousm damn good, rt peryiat go)",0.5059210868267039
0,"List(hmmmmm food religion, hassystants 12 816, fave look unbuttoned)",0.5046124569067216


In [0]:
top_ngrams_topics = get_top_overall_ngrams(topics, top_n=5)

display(top_ngrams_topics)

ngram
chanyeol sehun_chanyeol exo_sc
think babies mature
opinion httpstcosicxdyj4rc rt
top bgt masih
pickup ssr3 rt


## Final function
This final function integrates all the previous functions used to detect trending topics.

We get the necessary data, treat it and apply the topic detection algorithm

In [0]:
def final_function(start_year=2019, start_month=8, start_day=1, start_hour=2, start_minute=0, 
                   minutes_length=5, 
                   n=3,
                   num_clusters = 5,
                   special_characters=False,
                   time_window='1 minutes', 
                   apply_ner=True,
                   n_top_clusters=3,
                   n_top_ngrams=5):
    
    """

    Final function that retrieves the tweets, preprocesses them, and applies the bngram algorithm to extract topics (scored n-grams in their clusters).

    Parameters:
    -----------
    start_year (int): Starting year of the time range for the tweets being fetched (Default = 2019).
    start_month (int): Starting month of the time range for the tweets being fetched (Default = 8).
    start_day (int): Starting day of the time range for the tweets being fetched (Default = 1).
    start_hour (int): Starting hour of the time range for the tweets being fetched (Default = 2).
    start_minute (int): Starting minute of the time range for the tweets being fetched (Default = 0).
    minutes_length (int): Length of the time window in minutes to fetch tweets (Default = 5).
    n (int): Size of n-grams (e.g., 2 for bigrams, 3 for trigrams) to extract from the text (Default = 3).
    num_clusters (int): Number of clusters to generate during clustering (Default = 5).
    special_characters (bool): Whether to include special characters such as ⁠ # ⁠ (hashtags) and ⁠ @ ⁠ (mentions) in the filtering (Default = False).
    time_window (str): Aggregation window size (e.g., '5 minutes') for preprocessing tweets (Default = '1 minutes').
    apply_ner (bool): Whether to apply Named Entity Recognition (NER) and boost named entities during processing (Default = True).
    n_top_clusters (int): Number of top ngrmas per clusters to extract results from the final ranked results (Default = 3).
    n_top_ngrams (int): Number of top n-grams to extract as results from final ranked results (Default = 5).

    Returns:
    --------
    result_df (DataFrame): A Spark DataFrame containing the results of the bngram algorithm, including clusters and n-grams.
    top_topics_per_cluster (DataFrame): A Spark DataFrame containing the top n-grams for each of the clusters.
    top_ngrams_topics (DataFrame): A Spark DataFrame containing the top overall n-grams extracted from the dataset, over all clusters.

    """
  
    # Fetch tweets
    tweets = get_tweets(start_year=start_year, start_month=start_month, start_day=start_day, start_hour=start_hour, 
                        start_minute=start_minute, minutes_length=minutes_length)
    
    # Handle case where no tweets are fetched
    if tweets is None:
        print("No tweets fetched for the given time range.")
        return None

    # Cleaning
    cleaned_tweets = inital_cleaning_prep(tweets, special_characters=special_characters)

    # Filter English tweets
    english_tweets = filter_english_tweets(cleaned_tweets)

    # Preprocess tweets for bngram
    aggregated_tweets = preprocess_for_bngram(english_tweets, time_window=time_window, apply_ner=apply_ner)

    # Run the bngram algorithm
    result_df = bngram_algorithm(aggregated_tweets, n=n, apply_ner=apply_ner, num_clusters=num_clusters)

    # Get top ngrams per cluster
    top_topics_per_cluster = get_top_ngrams_per_cluster(result_df, top_n=n_top_clusters)

    # Get top overall ngrams
    top_ngrams_topics = get_top_overall_ngrams(result_df, top_n=n_top_ngrams)
  
    return result_df, top_topics_per_cluster, top_ngrams_topics

In [0]:
result, top_clusters, top_ngrams = final_function(start_year=2019, start_month=8, start_day=1, start_hour=2, start_minute=0, 
                   minutes_length=5, 
                   n=5,
                   num_clusters = 5,
                   special_characters=False,
                   time_window='1 minutes', 
                   apply_ner=False,
                   n_top_clusters=3,
                   n_top_ngrams=5)
result.show()


ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
[OK!]
+-------+--------------------+------------------+
|cluster|              ngrams|     average_score|
+-------+--------------------+------------------+
|      2|[chanyeol sehun_c...|0.5077807896778362|
|      4|[opened gate http...| 0.507086928801416|
|      1|[way overnight we...|0.5063540922014329|
|      3|[chanyeol sehun_c...|0.5059210868267039|
|      0|[hmmmmm food reli...|0.5046124569067216|
+-------+--------------------+------------------+



In [0]:
display(top_clusters)

cluster,top_ngrams,average_score
2,"List(chanyeol sehun_chanyeol exo_sc, think babies mature, opinion httpstcosicxdyj4rc rt)",0.5077807896778362
4,"List(opened gate httpstcoyzlabbgvvb, shouldnt tell someo, 20s baby life)",0.507086928801416
1,"List(way overnight week, face dig face, britains industry gre)",0.5063540922014329
3,"List(chanyeol sehun_chanyeol exo_sc, karlousm damn good, rt peryiat go)",0.5059210868267039
0,"List(hmmmmm food religion, hassystants 12 816, fave look unbuttoned)",0.5046124569067216


In [0]:
display(top_ngrams)

ngram
chanyeol sehun_chanyeol exo_sc
think babies mature
opinion httpstcosicxdyj4rc rt
top bgt masih
pickup ssr3 rt


It is also important to note that the algorithm can also be improved by creating ngrams of several sizes, of 3,4,5,6.. which may improve the results of the scores, the issue is the time it takes to run this, and that we do not have a list of ground truth topics to compare the results to. 

Like this issue, a lot of other factors could be improved, if we had something to verify our resuls with, like for instance the number of clusters, maybe event the time wnidows (which they vary in the paper) a lot of combinatiosn could be tested to see what seems to work best and when, but unfourtunelty it is not the case.

Aditionally, with more computational power and more time, a more refined version may have been implemented, since the time it takes to run is also a huge limitation.




## Conclusion

We started this project by reading the paper "Sensing Trending Topics in Twitter" which explores different algorithms for detecting trending topics in socal media. After reviwing these methods we decided that our best option was to implement the BNgram algorithm because of its performance on the paper and also its ability to capture patters of trending topics through n-gram analysis. 

Throughout this project, we collected the data by retrieving tweets using specified time windows. We performed preprocessing where we cleaned and filtered tweets, processed into tokenized ngrams. Using the BNgram algorithm, ngrams were evaluated based on their occurrance within and outside defined time window to compute relevance scores.

In addition, NER was optionally applied to enhance scores for the  eaniing entities, followed by clustering ngrrams into topics. Finally, clusters were ranked bassed on the average scores to identify the most relevant trends. All of this was incorporated in the final function which allows flexibility in parameters such as cluster number or the inclusion of special characters.

While we consider that the implementation was successful, we believe there are many areas of improvement. Firt, the algorithm was tested primarily with fixed ngram sizes. Expanding to multiple sizes could increase the diversity of captured patters. However, computational time made this difficult to implement. Moreover, The time requiered for ngrams, NER boosting and clustering proved to be a bottleneck. If we had more computational power larger datasets could be processed and more complex configurations could be tested to refine our results. During this project we tested different variations in parameters such as including/excluding special characters (@,#), applying NER, and using different window sizes.

Despite the challenges, we believe that our project successfully demonstrates trending topic detection with the BNgram algorithm. For futere work improvement we could use multiple ngram sizes to improve relevance scoring and do validation using a ground truth dataset for more reliable benchmarking.

In conclusion, this project successfully demonstrates the capability of the BNgram algorithm to process large scale social media data and uncover meaningful patterns in real time.